In [3]:
import cv2
import numpy as np

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Function to calculate speed
def calculate_speed(prev_pts, curr_pts, fps):
    # Calculate the displacement (difference in position)
    displacement = np.linalg.norm(curr_pts - prev_pts, axis=1)
    # Calculate speed (distance per frame) and convert to more meaningful units if needed
    speed = displacement * fps
    return speed

# Path to the video file
video_path = "C:/Users/gokul/Desktop/New folder (2)/vecteezy_the-soccer-ball-is-bouncing-in-slow-motion-for-video_39850756.mp4"

# Initialize video capture
cap = cv2.VideoCapture(video_path)

# Take the first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
prev_pts = cv2.goodFeaturesToTrack(old_gray, mask=None, **dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7))

fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    curr_pts, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, prev_pts, None, **lk_params)

    # Select good points
    good_new = curr_pts[st == 1]
    good_old = prev_pts[st == 1]

    # Calculate speed
    speeds = calculate_speed(good_old, good_new, fps)
    average_speed = np.mean(speeds)
    print(f"Average speed: {average_speed:.2f} units per second")

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        frame = cv2.line(frame, (a, b), (c, d), (0, 255, 0), 2)
        frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

    cv2.imshow('frame', frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    prev_pts = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()


Average speed: 34.78 units per second


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
